In [19]:
import rospy
import json
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from trajectory_utils import filter_joint_states
from std_msgs.msg import Float64

class JointStatePublisher:
    def __init__(self, input_file):
        self.input_file = input_file
        
        self.topic_name_right = "/yumi/joint_traj_pos_controller_r/command"

        self.gripper_right_topic = "/yumi/gripper_r_effort_cmd"
        self.gripper_left_topic = "/yumi/gripper_l_effort_cmd"
        
        # Read and filter joint states from file
        self.joint_states = filter_joint_states(self.read_from_file(), 0.01)
        
        self.publisher_right = rospy.Publisher(self.topic_name_right, JointTrajectory, queue_size=10)
        
        # Initialize publishers for grippers
        self.gripper_right_publisher = rospy.Publisher(self.gripper_right_topic, Float64, queue_size=10)
        self.gripper_left_publisher = rospy.Publisher(self.gripper_left_topic, Float64, queue_size=10)

    def read_from_file(self):
        try:
            with open(self.input_file, "r") as file:
                return json.load(file)
        except IOError as e:
            rospy.logerr(f"Failed to read from file: {e}")
            return []

    def publish_joint_states(self):
        rospy.init_node('yumi_joint_state_publisher', anonymous=True)
        rate = rospy.Rate(15)  # Define loop rate (10 Hz)

        past_gripper_status = 'open'
        while not rospy.is_shutdown():
            for i, joint_state in enumerate(self.joint_states):

                # Publish for right arm
                joint_traj_right = self.create_joint_trajectory(joint_state, '_r')
                self.publisher_right.publish(joint_traj_right)

                gripper_status = joint_state["gripper_status"]
                
                self.gripper_left_publisher.publish(20.0)
                self.gripper_right_publisher.publish(-20.0 if gripper_status == 'open' else 20.0)

                # Check if there's a change in gripper states compared to the past states
                if past_gripper_status!= gripper_status:
                    rospy.sleep(3)  # Delay if gripper states have changed


                # Update past_gripper_states for the next iteration
                past_gripper_status = gripper_status
                rospy.loginfo(f"Published joint state {i}")
                rate.sleep()
            break

    def create_joint_trajectory(self, joint_state, arm_suffix):
        """Helper method to create a joint trajectory for a given arm."""
        joint_traj = JointTrajectory()
        joint_traj.header.stamp = rospy.Time.now()
        joint_traj.joint_names = [name for name in joint_state["name"] if arm_suffix in name]
        
        point = JointTrajectoryPoint()
        point.positions = [joint_state["position"][i] for i, name in enumerate(joint_state["name"]) if arm_suffix in name]
        point.velocities = [joint_state["velocity"][i] for i, name in enumerate(joint_state["name"]) if arm_suffix in name]
        point.effort = [joint_state["effort"][i] for i, name in enumerate(joint_state["name"]) if arm_suffix in name]
        point.time_from_start = rospy.Duration(0.2)
        
        joint_traj.points = [point]
        return joint_traj

if __name__ == '__main__':
    input_file = "data/split_lego/right.json"  # Ensure correct path
    joint_state_publisher = JointStatePublisher(input_file)
    joint_state_publisher.publish_joint_states()

[INFO] [1707416790.381016]: Published joint state 0
[INFO] [1707416790.431154]: Published joint state 1
[INFO] [1707416790.481170]: Published joint state 2
[INFO] [1707416790.531165]: Published joint state 3
[INFO] [1707416790.581162]: Published joint state 4
[INFO] [1707416790.631195]: Published joint state 5
[INFO] [1707416790.681242]: Published joint state 6
[INFO] [1707416790.731154]: Published joint state 7
[INFO] [1707416790.781206]: Published joint state 8
[INFO] [1707416790.831146]: Published joint state 9
[INFO] [1707416790.881236]: Published joint state 10
[INFO] [1707416790.931146]: Published joint state 11
[INFO] [1707416790.981236]: Published joint state 12
[INFO] [1707416791.031160]: Published joint state 13
[INFO] [1707416791.081187]: Published joint state 14
[INFO] [1707416791.131478]: Published joint state 15
[INFO] [1707416791.181240]: Published joint state 16
[INFO] [1707416791.231182]: Published joint state 17
[INFO] [1707416791.281174]: Published joint state 18
[IN